In [ ]:
from tensorflow import keras
from imutils import paths
import glob
import matplotlib.pyplot as plt
import tensorflow as tf
import imageio
import h5py
from imutils import paths
from tqdm import tqdm
import pandas as pd 
import numpy as np
import shutil
import cv2
import os
from numba import jit
import math
import time

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!wget "http://people.csail.mit.edu/yalesong/tvsum/tvsum50_ver_1_1.tgz"

In [ ]:
def build_feature_extractor():
    feature_extractor = keras.applications.VGG19  (
        weights="imagenet",
        include_top=False,
        pooling="avg",
        input_shape=(224, 224, 3),
    )
    preprocess_input = keras.applications.vgg19.preprocess_input

    inputs = keras.Input((224, 224, 3))
    preprocessed = preprocess_input(inputs)

    outputs = feature_extractor(preprocessed)
    return keras.Model(inputs, outputs, name="feature_extractor")


feature_extractor = build_feature_extractor()

In [ ]:
@jit(nopython=True)
def scatters_with_numba(scatters,K1,K2,n):
  for i in range(0,n):
    for j in range(i,n):
      scatters[i, j] = K1[j+1]-K1[i] - (K2[j+1, j+1]+K2[i, i]-K2[j+1,i]-K2[i,j+1])/(j-i+1)
  
  return scatters

def calc_scatters(K):
    """
    Calculate scatter matrix:
    scatters[i,j] = {scatter of the sequence with starting frame i and ending frame j} 
    """
    n = K.shape[0]
    K1 = np.cumsum([0] + list(np.diag(K)))
    K2 = np.zeros((n+1, n+1))
    K2[1:, 1:] = np.cumsum(np.cumsum(K, 0), 1); # TODO: use the fact that K - symmetric

    scatters = np.zeros((n, n));
    
    scatters = scatters_with_numba(scatters,K1,K2,n)
    return scatters

@jit(nopython=True)
def max(x,y):
  if x > y:
    return x
  else :
    return y

@jit(nopython=True)
def cpd_nonlin_with_numba(m,n,p,I, J, lmin, lmax, backtrack):
  if backtrack == True:
    backtrack = 1
  else:
    backtrack = 0

  for k in range(1, m+1):
    for l in range((k+1)*lmin, n+1):
      I[k,l] = 1e100
      for t in range(max(k*lmin,l-lmax),l-lmin+1):
        c = I[k-1, t] + J[t, l-1]
        if c < I[k][l]:
          I[k, l] = c
          if backtrack == 1:
            p[k, l] = t


  return p,I


def cpd_nonlin(K, ncp, lmin=1, lmax=100000, backtrack=True, verbose=True,
    out_scatters=None):
    """ Change point detection with dynamic programming
    K - square kernel matrix 
    ncp - number of change points to detect (ncp >= 0)
    lmin - minimal length of a segment
    lmax - maximal length of a segment
    backtrack - when False - only evaluate objective scores (to save memory)
    
    Returns: (cps, obj)
        cps - detected array of change points: mean is thought to be constant on [ cps[i], cps[i+1] )    
        obj_vals - values of the objective function for 0..m changepoints
        
    """   
    m = int(ncp)  # prevent numpy.int64
    
    (n, n1) = K.shape
    assert(n == n1), "Kernel matrix awaited."    
    
    assert(n >= (m + 1)*lmin)
    assert(n <= (m + 1)*lmax)
    assert(lmax >= lmin >= 1)
    
    if verbose:
        #print "n =", n
        print ("Precomputing scatters...")
    J = calc_scatters(K)

    print(backtrack)
    
    if out_scatters != None:
        out_scatters[0] = J

    if verbose:
        print ("Inferring best change points...")
    # I[k, l] - value of the objective for k change-points and l first frames
    I = 1e101*np.ones((m+1, n+1))
    I[0, lmin:lmax] = J[0, lmin-1:lmax-1]

    if backtrack:
        # p[k, l] --- "previous change" --- best t[k] when t[k+1] equals l
        p = np.zeros((m+1, n+1), dtype=int)
    else:
        p = np.zeros((1,1), dtype=int)
            
    p,I = cpd_nonlin_with_numba(m,n,p,I, J, lmin, lmax, backtrack)
    
    # Collect change points
    cps = np.zeros(m, dtype=int)
    
    if backtrack:
        cur = n
        for k in range(m, 0, -1):
            cps[k-1] = p[k, cur]
            cur = cps[k-1]

    scores = I[:, n].copy() 
    scores[scores > 1e99] = np.inf
    return cps, scores
    

In [ ]:
def cpd_auto(K, ncp, vmax, desc_rate=1, **kwargs):
  m = ncp
  (_, scores) = cpd_nonlin(K, m, backtrack=False, **kwargs)
  
  N = K.shape[0]
  N2 = N*desc_rate  # length of the video before subsampling
  
  penalties = np.zeros(m+1)
  # Prevent division by zero (in case of 0 changes)
  ncp = np.arange(1, m+1)
  penalties[1:] = (vmax*ncp/(2.0*N2))*(np.log(float(N2)/ncp)+1)
  
  costs = scores/float(N) + penalties
  m_best = np.argmin(costs)
  (cps, scores2) = cpd_nonlin(K, m_best, **kwargs)

  return (cps, costs)

In [ ]:
class Data_Generator:
  """
  vgg19 modelini indirmeden önce frame işlemleri yaklaşık 200 saniye sürdü
  indirdikten sonra ise yaklaşık 13 saniye sürdü

  change point işlemleri : 0.0063135623931884766 saniye sürdü

  """
  def __init__(self,video_path,save_path):
    self.dataset = {}
    self.video_list = []
    self.video_path = ""
    self.frame_root_path = '/content/drive/MyDrive/summe/frames'
    self.h5_file = h5py.File(save_path, 'w')
    self.set_video_list(video_path,save_path)


  def set_video_list(self, video_path,save_path):
    if os.path.isdir(video_path):
      self.video_path = video_path
      files = glob.glob(video_path+"/*.mp4", recursive = True)
      self.video_list = files
      self.video_list.sort()
    
    else:
      self.video_path = ""
      self.video_list.append(video_path)
    
    for idx, file_name in enumerate(self.video_list):
      self.dataset["video_{}".format(idx+1)] = {}
      self.h5_file.create_group("video_{}".format(idx+1))


  def extract_feature(self, frame):
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frame = cv2.resize(frame, (224, 224))
    frame = frame[None, ...]
    feature = feature_extractor.predict(frame)
    frame_feature = feature.flatten()

    return feature

  def get_change_point(self, video_feat, n_frame, fps):
    n = n_frame / fps
    m = int(math.ceil(n/2.0))
    K = np.dot(video_feat, video_feat.T)
    change_points, _ = cpd_auto(K, m, 1)
    change_points = np.concatenate(([0], change_points, [n_frame-1]))

    temp_change_points = []
    for idx in range(len(change_points)-1):
        segment = [change_points[idx], change_points[idx+1]-1]
        if idx == len(change_points)-2:
            segment = [change_points[idx], change_points[idx+1]]

        temp_change_points.append(segment)
    change_points = np.array(list(temp_change_points))

    temp_n_frame_per_seg = []
    for change_points_idx in range(len(change_points)):
        n_frame = change_points[change_points_idx][1] - change_points[change_points_idx][0]
        temp_n_frame_per_seg.append(n_frame)
    n_frame_per_seg = np.array(list(temp_n_frame_per_seg))

    return change_points, n_frame_per_seg


  def generate_ds(self):
    for video_idx, video_filename in enumerate(tqdm(self.video_list)):
      video_path = video_filename
      if os.path.isdir(self.video_path):
        video_path = os.path.join(self.video_path, video_filename)
      video_basename = os.path.basename(video_path).split(".")[0]

      if not os.path.exists(os.path.join(self.frame_root_path, video_basename)):
        os.mkdir(os.path.join(self.frame_root_path, video_basename))

      video_capture = cv2.VideoCapture(video_path)
      fps = video_capture.get(cv2.CAP_PROP_FPS)
      n_frames = int(video_capture.get(cv2.CAP_PROP_FRAME_COUNT))

      frame_list = []
      picks = []
      video_feat = None
      video_feat_for_train = None

      start_time = time.time()
      
      for frame_idx in range(n_frames-1):
        success, frame = video_capture.read()
        if success:
            frame_feat = self.extract_feature(frame)

            if frame_idx % 15 == 0:
                picks.append(frame_idx)

                if video_feat_for_train is None:
                    video_feat_for_train = frame_feat
                else:
                    video_feat_for_train = np.vstack((video_feat_for_train, frame_feat))

            if video_feat is None:
                video_feat = frame_feat
            else:
                video_feat = np.vstack((video_feat, frame_feat))

            img_filename = "{}.jpg".format(str(frame_idx).zfill(5))
            cv2.imwrite(os.path.join(self.frame_root_path, video_basename, img_filename), frame)

        else:
            break

      video_capture.release()
      print("\n The passing time for frames :",time.time()-start_time)
      
      start_time = time.time() 

      change_points, n_frame_per_seg = self.get_change_point(video_feat, n_frames, fps)
      
      print("\n The passing time for frames :",time.time()-start_time)


      self.h5_file['video_{}'.format(video_idx+1)]['features'] = list(video_feat_for_train)
      self.h5_file['video_{}'.format(video_idx+1)]['picks'] = np.array(list(picks))
      self.h5_file['video_{}'.format(video_idx+1)]['n_frames'] = n_frames
      self.h5_file['video_{}'.format(video_idx+1)]['fps'] = fps
      self.h5_file['video_{}'.format(video_idx+1)]['change_points'] = change_points
      self.h5_file['video_{}'.format(video_idx+1)]['n_frame_per_seg'] = n_frame_per_seg

#Yeni Data Generator


In [ ]:
class Data_GeneratorNew:
  """
  10 saniyelik video için

  vgg19 modelini indirmeden önce frame işlemleri yaklaşık 200 saniye sürdü
  indirdikten sonra ise yaklaşık 13 saniye sürdü
  
  değişiklerden sonra bu süre 11-12 saniyeye düştü


  change point işlemleri : 0.0063135623931884766 saniye sürdü

  """
  def __init__(self,video_path,save_path):
    self.dataset = {}
    self.video_list = []
    self.video_path = ""
    self.frame_root_path = '/content/drive/MyDrive/summe/frames'
    self.h5_file = h5py.File(save_path, 'w')
    self.set_video_list(video_path,save_path)


  def set_video_list(self, video_path,save_path):
    if os.path.isdir(video_path):
      self.video_path = video_path
      files = glob.glob(video_path+"/*.mp4", recursive = True)
      self.video_list = files
      self.video_list.sort()
    
    else:
      self.video_path = ""
      self.video_list.append(video_path)
    
    for idx, file_name in enumerate(self.video_list):
      self.dataset["video_{}".format(idx+1)] = {}
      self.h5_file.create_group("video_{}".format(idx+1))

  def get_change_point(self, video_feat, n_frame, fps):
    n = n_frame / fps
    m = int(math.ceil(n/2.0))
    K = np.dot(video_feat, video_feat.T)
    change_points, _ = cpd_auto(K, m, 1)
    change_points = np.concatenate(([0], change_points, [n_frame-1]))

    temp_change_points = []
    for idx in range(len(change_points)-1):
        segment = [change_points[idx], change_points[idx+1]-1]
        if idx == len(change_points)-2:
            segment = [change_points[idx], change_points[idx+1]]

        temp_change_points.append(segment)
    change_points = np.array(list(temp_change_points))

    temp_n_frame_per_seg = []
    for change_points_idx in range(len(change_points)):
        n_frame = change_points[change_points_idx][1] - change_points[change_points_idx][0]
        temp_n_frame_per_seg.append(n_frame)
    n_frame_per_seg = np.array(list(temp_n_frame_per_seg))

    return change_points, n_frame_per_seg


  def generate_ds(self):
    for video_idx, video_filename in enumerate(self.video_list):
      video_path = video_filename
      if os.path.isdir(self.video_path):
        video_path = os.path.join(self.video_path, video_filename)
      video_basename = os.path.basename(video_path).split(".")[0]

      if not os.path.exists(os.path.join(self.frame_root_path, video_basename)):
        os.mkdir(os.path.join(self.frame_root_path, video_basename))

      video_capture = cv2.VideoCapture(video_path)
      fps = video_capture.get(cv2.CAP_PROP_FPS)
      n_frames = int(video_capture.get(cv2.CAP_PROP_FRAME_COUNT))

      n_train_frames = int(n_frames / 15) +1
      count_train = 0 
      count_feat = 0
      picks = []
      video_feat = np.zeros(shape = (n_frames-n_train_frames,512))
      video_feat_for_train = np.zeros(shape = (n_train_frames,512))

      for frame_idx in range(n_frames-1):
        success, frame = video_capture.read()
        img_filename = "{}.jpg".format(str(frame_idx).zfill(5))
        cv2.imwrite(os.path.join(self.frame_root_path, video_basename, img_filename), frame)
        if success:
          if frame_idx % 15 == 0:
            picks.append(frame_idx)
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frame = cv2.resize(frame, (224, 224))
            frame = frame[None, ...]
            feature = feature_extractor.predict(frame)
            feature = feature.flatten()
            video_feat_for_train[count_train] = feature
            count_train += 1
          else:
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frame = cv2.resize(frame, (224, 224))
            frame = frame[None, ...]
            feature = feature_extractor.predict(frame)
            feature = feature.flatten()
            video_feat[count_feat] = feature
            count_feat += 1
        else:
          break

      video_capture.release()      

      change_points, n_frame_per_seg = self.get_change_point(video_feat, n_frames, fps)
      
      self.h5_file['video_{}'.format(video_idx+1)]['features'] = list(video_feat_for_train)
      self.h5_file['video_{}'.format(video_idx+1)]['picks'] = np.array(list(picks))
      self.h5_file['video_{}'.format(video_idx+1)]['n_frames'] = n_frames
      self.h5_file['video_{}'.format(video_idx+1)]['fps'] = fps
      self.h5_file['video_{}'.format(video_idx+1)]['change_points'] = change_points
      self.h5_file['video_{}'.format(video_idx+1)]['n_frame_per_seg'] = n_frame_per_seg


In [ ]:
if __name__ == "__main__":
    gen = Data_GeneratorNew('/content/drive/MyDrive/TVSUM/video', '/content/drive/MyDrive/TVSum.h5')
    gen.generate_ds()
    gen.h5_file.close()